In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import time

from bs4 import BeautifulSoup as bs
import pandas as pd
import json
from progressbar import progressbar as pb

import sys
sys.path.append("../../../experiments/") # use utils

import warnings
warnings.filterwarnings("ignore")

In [ ]:
url = "https://sites.research.google/biomedexplorer/"
pubmed_re = re.compile("https://www.ncbi.nlm.nih.gov/pubmed/\d+")

In [ ]:
df = pd.read_csv("../../../data/data_to_process.csv")
print(len(df))
df.head(3)

In [ ]:
def get_pubmed_ids_from_explorer(query, url=url, sleep=5, pubmed_re=pubmed_re):
    caps = DesiredCapabilities().CHROME
    caps["pageLoadStrategy"] = "eager"
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("window-size=1200x600")
    
    driver = webdriver.Chrome("selenium/chromedriver", 
                                      desired_capabilities=caps,
                                      options=chrome_options
                             )
    driver.get(url)
    zoom = driver.find_element(By.XPATH, "//input[@placeholder='Ask a question like \"What regulates the Warburg effect\"']")
    zoom.click()
    zoom.send_keys(query)
    zoom.send_keys(Keys.RETURN)
    time.sleep(sleep)
    html_page = driver.page_source
    driver.close()
    
    return [item.split("/")[-1] for item in pubmed_re.findall(html_page)]

## Keywords

In [ ]:
result = {source: {} for source in df.data_source.unique()}

for _, info in pb(df.iterrows(), max_value=len(df)):
    d_source = info.data_source
    q_id = info.query_id
    q = info.query
    
    pubmed_ids = get_pubmed_ids_from_explorer(q)
    
    result[d_source][q_id] = pubmed_ids
    time.sleep(2)

In [ ]:
with open('biomed_explorer_abstracts_keywords.json', 'w') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

## Question

In [ ]:
result = {source: {} for source in df.data_source.unique()}

for _, info in pb(df.iterrows(), max_value=len(df)):
    d_source = info.data_source
    q_id = info.query_id
    q = info.description
    
    pubmed_ids = get_pubmed_ids_from_explorer(q)
    
    result[d_source][q_id] = pubmed_ids
    time.sleep(2)

In [ ]:
with open('biomed_explorer_abstracts_question.json', 'w') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

## Make CSV's

In [ ]:
with open("../../../data/doc_id2abstract.json") as f:
    doc_id2abstract = json.load(f)

## Keywords

In [ ]:
with open("biomed_explorer_abstracts_keywords.json") as f:
    query_info = json.load(f)

In [ ]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id", "abstract"]}

for data_source, item in query_info.items():
    for query_id, doc_ids in item.items():
        for doc_id in doc_ids:
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)
            for_dataframe['abstract'].append(doc_id2abstract[doc_id])

In [ ]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(df, on=["query_id", "data_source"])
)
fin_df = fin_df.assign(query_processed=fin_df.description)

In [ ]:
fin_df.to_csv("../../../data/biomed_explorer/biomed_explorer_abstracts_keywords.csv", index=0)

## Question

In [ ]:
with open("biomed_explorer_abstracts_question.json") as f:
    query_info = json.load(f)

In [ ]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id", "abstract"]}

for data_source, item in descr_info.items():
    for query_id, doc_ids in item.items():
        for doc_id in doc_ids:
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)
            for_dataframe['abstract'].append(doc_id2abstract[doc_id])

In [ ]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(df, on=["query_id", "data_source"])
)
fin_df = fin_df.assign(query_processed=fin_df.description)

In [ ]:
fin_df.to_csv("../../../data/biomed_explorer/biomed_explorer_abstracts_question.csv", index=0)